In [1]:
%%capture
! pip install ml-collections

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/DeepGaze

In [2]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import random
import copy

import os
import shutil
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import face
from scipy.ndimage import zoom
from scipy.special import logsumexp
import torch
import deepgaze_pytorch
from deepgaze_pytorch import modules

from coco.dataset import COCODataset
from utils import *
sys.path.append("..")

In [3]:
def logsearchProcess(x, y, tg_xy, attentionMap, image_size, size, coef):
    mask_size = size
    tg_x, tg_y, w, h = tg_xy
    tg_xmax, tg_ymax = tg_x + w, tg_y + h 

    attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
    y_fix, x_fix = y, x

    x_max_s, x_min_s, y_max_s, y_min_s = min(x_fix+mask_size//2, image_size[1]-1), max(x_fix-mask_size//2, 0), min(y_fix+mask_size//2, image_size[0]-1), max(y_fix-mask_size//2, 0)

    if x_max_s < tg_x or x_min_s > tg_xmax or y_max_s < tg_y or y_min_s > tg_ymax:
        coef[0, y_min_s:y_max_s+1, x_min_s:x_max_s+1] = 1000
        attenNP = (attentionMap[0,:,:].detach() * coef[0,:,:].detach()).numpy()
        y_fix, x_fix = np.unravel_index(attenNP.argmax(), attenNP.shape)
        return False, (x_fix, y_fix), coef

    return True, [], coef

In [4]:
%%capture
test_annotations = "../../debug/coco18_test.json"
test_imagedir = "../../coco18_test"
image_list = os.listdir(test_imagedir)
input_images = COCODataset(test_annotations, test_imagedir, image_size=(224,224))

read in test dataset

In [5]:
coco18_test = "../../coco_search18_fixations_TP_test.json"
df_test = pd.read_json(coco18_test)
df_test.head()

,name,subject,task,condition,bbox,X,Y,T,length,fixOnTarget,correct,split
0,000000394517.jpg,2,bottle,present,"[1348, 559, 142, 395]","[910.9, 1194.5, 1445.0]","[338.6, 670.3, 684.5]","[203, 79, 178, 294, 372, 362, 184]",3,True,1,test
1,000000276488.jpg,2,bottle,present,"[560, 0, 322, 417]","[847.8, 880.4, 700.2]","[530.9, 454.6, 166.0]","[171, 141, 231, 174, 198]",3,True,1,test
2,000000182213.jpg,2,bottle,present,"[128, 255, 246, 513]","[843.2, 773.5, 529.7, 333.5]","[519.0, 435.4, 417.7, 516.8]","[57, 188, 110, 148, 165, 286, 518]",4,True,1,test
3,000000371864.jpg,2,bottle,present,"[708, 84, 112, 282]","[770.7, 957.1, 824.5, 794.7]","[388.3, 314.7, 270.9, 202.4]","[222, 49, 145, 365, 174, 305, 291, 209, 159]",4,True,1,test
4,000000225129.jpg,2,bottle,present,"[893, 0, 126, 173]","[839.0, 743.2, 948.2]","[506.0, 475.5, 162.1]","[55, 266, 175, 512, 219]",3,True,1,test


In [6]:
df_test[df_test.length == 1]

,name,subject,task,condition,bbox,X,Y,T,length,fixOnTarget,correct,split
326,000000576667.jpg,2,toilet,present,"[857, 678, 465, 354]",[777.8],[694.1],[223],1,False,1,test
394,000000492905.jpg,1,tv,present,"[708, 88, 318, 302]",[878.9],[372.3],"[251, 278, 473]",1,True,1,test
473,000000279850.jpg,1,car,present,"[53, 722, 164, 181]",[829.4],[516.4],[274],1,False,1,test
507,000000287443.jpg,2,chair,present,"[575, 0, 575, 122]",[865.8],[518.4],[515],1,False,0,test
618,000000448897.jpg,1,cup,present,"[172, 0, 249, 277]",[882.1],[468.8],[240],1,False,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...
5950,000000195754.jpg,4,tv,present,"[1155, 338, 234, 227]",[749.4],[512.6],[226],1,False,1,test
5953,000000085940.jpg,4,tv,present,"[170, 317, 428, 343]",[799.3],[462.8],[238],1,False,1,test
5954,000000576070.jpg,4,tv,present,"[1325, 312, 185, 176]",[817.4],[548.5],[229],1,False,1,test
6009,000000044263.jpg,8,tv,present,"[1181, 490, 334, 297]",[847.2],[463.1],[180],1,False,1,test


In [7]:
df_test[(df_test.name=='000000488201.jpg') & (df_test.task == 'tv')]

,name,subject,task,condition,bbox,X,Y,T,length,fixOnTarget,correct,split
368,000000488201.jpg,1,tv,present,"[535, 63, 448, 336]","[986.2, 917.9, 728.8]","[467.4, 462.6, 266.4]","[92, 108, 365, 417]",3,True,1,test
1090,000000488201.jpg,2,tv,present,"[535, 63, 448, 336]","[728.2, 784.3]","[459.3, 315.6]","[244, 236, 215]",2,True,1,test
2370,000000488201.jpg,5,tv,present,"[535, 63, 448, 336]",[626.5],[312.4],"[188, 203, 738]",1,True,1,test
2401,000000488201.jpg,6,tv,present,"[535, 63, 448, 336]","[834.1, 781.6]","[552.8, 348.8]","[198, 317, 221, 580]",2,True,1,test
3025,000000488201.jpg,3,tv,present,"[535, 63, 448, 336]","[801.0, 781.4]","[479.5, 245.2]","[283, 374, 459]",2,True,1,test
5882,000000488201.jpg,10,tv,present,"[535, 63, 448, 336]","[711.1, 709.0]","[436.1, 264.1]","[209, 585]",2,True,1,test
5910,000000488201.jpg,4,tv,present,"[535, 63, 448, 336]","[853.5, 805.3, 762.5]","[482.5, 426.5, 241.9]","[246, 134, 180]",3,True,1,test
6010,000000488201.jpg,8,tv,present,"[535, 63, 448, 336]","[827.9, 784.9]","[454.1, 321.1]","[243, 338, 229]",2,True,1,test
6035,000000488201.jpg,9,tv,present,"[535, 63, 448, 336]","[844.7, 723.8]","[526.5, 291.9]","[78, 311, 235, 180, 28]",2,True,1,test
6094,000000488201.jpg,7,tv,present,"[535, 63, 448, 336]",[848.5],[397.9],"[214, 220, 342]",1,True,1,test


In [16]:
img_size = (320, 512)
tg_bbox_dic, img_initial_fixation = {}, {}
ratio_horizontal, ratio_vertical = 1680/512, 1050/320

for id in trange(len(input_images)):
    _, _, bbox, labels_cpu = input_images[id]
    label_name = input_images.idx2label[labels_cpu]
    
    file_name = input_images.id2file[input_images.annotations[id]['image_id']]
    file_name = file_name.split('/')[-1]
    if file_name[:len(label_name)] == label_name:
        file_num = file_name.split('/')[-1][len(label_name):]
    else:
        file_num = file_name.split('/')[-1]
        
    df_select = df_test[(df_test.name==file_num) & (df_test.task == label_name)]
    choice_list = list(range(len(df_select)))
    random_num = random.sample(choice_list, 3)
    x, y = [], []
    for num in random_num:
        sample = df_select.iloc[num, :]
        x.append(int(sample['X'][0]/ratio_horizontal))
        y.append(int(sample['Y'][0]/ratio_vertical))

    x.append(256)
    y.append(160)

    img_initial_fixation[file_name] = (x, y)

    tg_loc = bbox_cordinates(bbox, img_size[1], img_size[0])
    tg_bbox_dic[file_name] = tg_loc

100%|██████████| 612/612 [00:27<00:00, 22.58it/s]


In [19]:
%%capture
DEVICE = 'cuda'
# you can use DeepGazeI or DeepGazeIIE
model = deepgaze_pytorch.DeepGazeIII(pretrained=True).to(DEVICE)
image = face()
centerbias_template = np.load('centerbias_mit1003.npy')
# rescale to match image size
centerbias = zoom(centerbias_template, (image.shape[0]/centerbias_template.shape[0], image.shape[1]/centerbias_template.shape[1]), order=0, mode='nearest')
# renormalize log density
centerbias -= logsumexp(centerbias)
centerbias_tensor = torch.tensor([centerbias]).to(DEVICE)
centerbias_tensor = transforms.Resize(img_size)(centerbias_tensor)

In [20]:
deepgaze3_res, size, scanpath = [], 48, {}

for i in trange(len(image_list)):
    # get the target bounding box
    tg_loc = tg_bbox_dic[image_list[i]]
    history_x = img_initial_fixation[image_list[i]][0]
    history_y = img_initial_fixation[image_list[i]][1]

    # get the img path
    img_dir = os.path.join(test_imagedir, image_list[i]) 
    img = cv2.imread(img_dir)

    img_tensor = torch.tensor([img.transpose(2, 0, 1)]).to(DEVICE)
    img_tensor = transforms.Resize(img_size)(img_tensor)

    count, max_search, coef, path = 0, 999, torch.ones((1, 320, 512)), []
    while count < max_search:
        fixation_history_x = np.array(history_x)
        fixation_history_y = np.array(history_y)
        x_hist_tensor = torch.tensor([fixation_history_x[model.included_fixations]]).to(DEVICE)
        y_hist_tensor = torch.tensor([fixation_history_y[model.included_fixations]]).to(DEVICE)
        log_density_prediction = model(img_tensor, centerbias_tensor, x_hist_tensor, y_hist_tensor)

        path.append([history_x[-1], history_y[-1]])
        isTg, coordinates, coef = logsearchProcess(history_x[-1], history_y[-1], tg_loc, log_density_prediction.squeeze(0).cpu(), img_size, size, coef)
        count += 1

        if isTg: 
            break

        history_x.append(coordinates[0])
        history_y.append(coordinates[1])

    scanpath[image_list[i]] = path

    print(count)
    deepgaze3_res.append(count)


  0%|          | 1/612 [00:23<4:01:03, 23.67s/it]

105


  0%|          | 2/612 [00:31<2:24:27, 14.21s/it]

33


  0%|          | 3/612 [00:32<1:23:53,  8.27s/it]

...

36


100%|█████████▉| 611/612 [1:04:05<00:06,  6.93s/it]

21


100%|██████████| 612/612 [1:04:07<00:00,  6.29s/it]

6


In [ ]:
scanpath

In [22]:
deepgaze_res_fig5 = {}
deepgaze_res_fig5['scanpath'] = scanpath
deepgaze_res_fig5['search_list'] = deepgaze3_res

In [23]:
with open("../results/COCO/[Fig5]coco_deepgaze_res.pkl", "wb") as tf:
    pickle.dump(deepgaze_res_fig5, tf)

In [24]:
deepgaze3_accu_coco = [0] + model_performance(deepgaze3_res, len(deepgaze3_res))
deepgaze3_accu_coco[:11]

[0,
 0.0,
 0.06045751633986928,
 0.119281045751634,
 0.17483660130718953,
 0.23202614379084968,
 0.2565359477124183,
 0.27941176470588236,
 0.3104575163398693,
 0.3382352941176471,
 0.37254901960784315]

In [25]:
with open("../results/coco_deepgaze3_accu_performance.pkl", "wb") as tf:
    pickle.dump(deepgaze3_accu_coco, tf)

In [ ]:
#@title Sample Code { display-mode: "form" }
# location of previous scanpath fixations in x and y (pixel coordinates), starting with the initial fixation on the image.
fixation_history_x = np.array([1024//2, 300, 500, 1024//2, 200, 700])
fixation_history_y = np.array([768//2, 300, 100, 768//2, 100, 500])
x_hist_tensor = torch.tensor([fixation_history_x[model.included_fixations]]).to(DEVICE)
y_hist_tensor = torch.tensor([fixation_history_y[model.included_fixations]]).to(DEVICE)

image = face()
print(type(image))

# load precomputed centerbias log density (from MIT1003) over a 1024x1024 image
# you can download the centerbias from https://github.com/matthias-k/DeepGaze/releases/download/v1.0.0/centerbias_mit1003.npy
# alternatively, you can use a uniform centerbias via `centerbias_template = np.zeros((1024, 1024))`.
centerbias_template = np.load('centerbias_mit1003.npy')
# rescale to match image size
centerbias = zoom(centerbias_template, (image.shape[0]/centerbias_template.shape[0], image.shape[1]/centerbias_template.shape[1]), order=0, mode='nearest')
# renormalize log density
centerbias -= logsumexp(centerbias)
centerbias_tensor = torch.tensor([centerbias]).to(DEVICE)
image_tensor = torch.tensor([image.transpose(2, 0, 1)]).to(DEVICE)

f, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))
axs[0].imshow(image)
axs[0].plot(fixation_history_x, fixation_history_y, 'o-', color='red')
axs[0].scatter(fixation_history_x[-1], fixation_history_y[-1], 100, color='yellow', zorder=100)
axs[0].set_axis_off()
axs[1].matshow(log_density_prediction.detach().cpu().numpy()[0, 0])  # first image in batch, first (and only) channel
axs[1].plot(fixation_history_x, fixation_history_y, 'o-', color='red')
axs[1].scatter(fixation_history_x[-1], fixation_history_y[-1], 100, color='yellow', zorder=100)
axs[1].set_axis_off()

<class 'numpy.ndarray'>


torch.Size([1, 3, 320, 512])